#  **Testing Gaussian EP on a given RBM**


In [ ]:
using Pkg
using Plots
using Revise
using DelimitedFiles
using BenchmarkTools

In [ ]:
include("../src/GaussianEP.jl")
#using .GaussianEP
#include("../Sampling/Sampling.jl")
#Pkg.activate("/Users/luca/repos/GaussianEP/")
#using GaussianEP

In [ ]:
# Number of visible and hidden units, Gaussian weigths
N = 8
M = 3
w = randn(N,M)
y = zeros(N+M)
H = [GaussianEP.TermRBM(w,y,1.0)];

# (Pv, Ph) = (Gaussian - Gaussian) priors

In [ ]:
μv = 3.0; βv = 5.0;
μh = 5.0; βh = 4.0;

Pv = [GaussianEP.GaussianPrior(μv, βv, 0.0) for i = 1:N];
Ph = [GaussianEP.GaussianPrior(μh, βh, 0.0) for i = 1:M];


In [ ]:
out = GaussianEP.expectation_propagation(H, Pv, Ph, nprint = 100);

In [ ]:
using Plots

Σan = GaussianEP.compute_analytic(w,Pv,Ph)
p1 = heatmap(Σan, aspect_ratio=1, ticks= :false, showaxis = :false, title="True covariance matrix", titlelocation=:center)
p2 = heatmap(out.state.Σ, aspect_ratio=1, ticks= :false, showaxis =:false, title="EP covariance matrix", titlelocation=:center)
Plots.plot(p1,p2)

# (Pv, Ph) = (Binary - Gaussian) priors

In [ ]:
x0v = 0.0; x1v = 1.0; ρ = 0.9;
μh = 0.5; βh = 3.0; 
Pv = nothing; Ph = nothing;
Pv = [GaussianEP.BinaryPrior(x0v, x1v, ρ) for i = 1:N]
Ph = [GaussianEP.GaussianPrior(μh, βh, 0.0) for j = 1:M];


In [ ]:
out = GaussianEP.expectation_propagation(H, Pv, Ph, epsconv = 1e-7, maxiter=100000);
cov_ep = zeros(N,M)
for i = 1:N, j = 1:M
	cov_ep[i,j] = out.state.Σ[i,N+j] + out.state.av[i] * out.state.av[N+j]
end

In [ ]:
av_an, va_an, cov_an = GaussianEP.compute_analytic(w, Pv, Ph);

In [ ]:
p1 = heatmap(cov_an', aspect_ratio=1, ticks= :false, showaxis = :false, xlabel = "visible", ylabel = "hidden" ,title="True covariance matrix", titlelocation=:center)
p2 = heatmap(cov_ep', aspect_ratio=1, ticks= :false, xlabel = "visible", ylabel = "hidden", showaxis =:false,  title="EP covariance matrix", titlelocation=:center)
p3 = Plots.plot(av_an, av_an, label = "x = y") 
p3 = Plots.plot!(av_an, out.av, seriestype = :scatter, label = "EP means", legend = :topleft, xlabel = "True means")
p4 = Plots.plot(va_an, va_an, label = "x = y")
p4 = Plots.plot!(va_an, out.va, seriestype = :scatter, label = "EP vars", legend = :topleft, xlabel = "True var")
p5 = Plots.plot(vec(cov_an), vec(cov_an), label = "x = y")
p5 = Plots.plot!(vec(cov_an), vec(cov_ep), seriestype = :scatter, label = "EP cov", legend = :bottomright, xlabel = "True cross cov")
l = @layout [a b; c d e]
Plots.plot(p1,p2,p3, p4, p5, layout = l)

In [ ]:
Pv = [Sampling.BinaryPrior(x0v, x1v, ρ) for i = 1:N]
Ph = [Sampling.GaussianPrior(μh, βh, 0.0) for j = 1:M]
P0mc = vcat(Pv,Ph);

In [ ]:
# Get acceptance rate ~ 0.5
Δ_vec = collect(0.001:0.005:0.2)
@time acc_tab = Sampling.MC_Δ_scan(Δ_vec,w,P0mc,10^5)
plot(acc_tab[:,1], acc_tab[:,2], xlabel="Δ", ylabel="acc. rate", seriestype = :scatter)

In [ ]:
# Get thermalization time

Δ = 0.05
nblocks = 22
mc_energy = Sampling.MC_t_therm(nblocks, w, P0mc, Δ) 
@time bl_tab = Sampling.block_anal(nblocks, mc_energy)

p1 = plot(bl_tab[:,1], bl_tab[:,2], ribbon=bl_tab[:,3],fillalpha=.5, xscale =:log, xlabel="MC sweeps", ylabel="Energy") 
plot(p1)


In [ ]:
# Sampling using MCMC
bl_tab = nothing
Nconf = 10^6
Twait = 10^6
@time mc_out = Sampling.MC_sim(w, P0mc, Twait, Δ; N_iter=Nconf);

In [ ]:
# Compute statistics

@time av_mc, va_mc, cov_mc = Sampling.compute_statistics(mc_out, N, M);

In [ ]:
p1 = heatmap(cov_an', aspect_ratio=1, ticks= :false, showaxis = :false, xlabel = "visible", ylabel = "hidden" ,title="True covariance matrix", titlelocation=:center)
p2 = heatmap(cov_mc', aspect_ratio=1, ticks= :false, xlabel = "visible", ylabel = "hidden", showaxis =:false,  title="MC covariance matrix", titlelocation=:center)
p3 = Plots.plot(av_an, av_an, label = "x = y") 
p3 = Plots.plot!(av_an, av_mc, seriestype = :scatter, label = "MC means", legend = :topleft, xlabel = "True av")
p4 = Plots.plot(va_an, va_an, label = "x = y")
p4 = Plots.plot!(va_an, va_mc, seriestype = :scatter, label = "MC vars", legend = :topleft, xlabel = "True va")
p5 = Plots.plot(vec(cov_an), vec(cov_an), label = "x = y")
p5 = Plots.plot!(vec(cov_an), vec(cov_mc), seriestype = :scatter, label = "MC cov", legend = :bottomright, xlabel = "True cross cov")
l = @layout [a b; c d e]
Plots.plot(p1,p2,p3, p4, p5, layout = l)

# (Pv, Ph) = (Gaussian, ReLu) Priors

In [ ]:

# Number of visible and hidden units, Gaussian weigths
N = 100
M = 10
w = randn(N,M)
y = zeros(N+M)
H = [GaussianEP.TermRBM(w,y,1.0)];
γ = 50.0
θ = 1.0
μv = 0.0; βv = 10.0; # βv needs to be large

Pv = [GaussianEP.GaussianPrior(μv, βv, 0.0) for i = 1:N];
Ph = [GaussianEP.ReLUPrior(γ, θ, 1e-2, 1e-2) for j = 1:M ]


In [ ]:
out = GaussianEP.expectation_propagation(H, Pv, Ph, nprint = 100)

cov_ep = zeros(N,M)
for i = 1:N, j = 1:M
	cov_ep[i,j] = out.state.Σ[i,N+j] + out.state.av[i] * out.state.av[N+j]
end

In [ ]:
γ_m = mean([P0[N+i].γ for i = 1:M])
θ_m = mean([P0[N+i].θ for i = 1:M])

In [ ]:
Pv = [Sampling.GaussianPrior(μv, βv, 0.0) for i = 1:N];
Ph = [Sampling.ReLUPrior(γ_m, θ_m) for j = 1:M ]
P0mc = vcat(Pv,Ph);

In [ ]:
# Get acceptance rate ~ 0.5
Δ_vec = collect(0.05:0.05:0.9)
acc_tab = Sampling.MC_Δ_scan(Δ_vec,w,P0mc,10^3)
plot(acc_tab[:,1], acc_tab[:,2], xlabel="Δ", ylabel="acc. rate", seriestype = :scatter)

In [ ]:

# Get thermalization time

Δ = 0.1
nblocks = 20
mc_energy = Sampling.MC_t_therm(nblocks, w, P0mc, Δ) 
bl_tab = Sampling.block_anal(nblocks, mc_energy)

p1 = plot(bl_tab[:,1], bl_tab[:,2], ribbon=bl_tab[:,3],fillalpha=.5, xscale =:log) 
plot(p1)

In [ ]:
# Sampling using MCMC

mc_energy = nothing
bl_tab = nothing
mc_out = nothing

Δ = 0.1
Nconf = 5 * 10^5
Twait = 10^6

mc_out = Sampling.MC_sim(w, P0mc, Twait, Δ; N_iter=Nconf);

In [ ]:
# Compute statistics

av_mc, va_mc, cov_mc = Sampling.compute_statistics(mc_out, N, M);

In [ ]:

p1 = heatmap(cov_ep', aspect_ratio=1, ticks= :false, showaxis = :false, xlabel = "visible", ylabel = "hidden" ,title="EP covariance matrix", titlelocation=:center)
p2 = heatmap(cov_mc', aspect_ratio=1, ticks= :false, xlabel = "visible", ylabel = "hidden", showaxis =:false,  title="MC covariance matrix", titlelocation=:center)
p3 = Plots.plot(av_mc, av_mc, label = "x = y") 
p3 = Plots.plot!(av_mc, out.av, seriestype = :scatter, label = "EP means", legend = :topleft, xlabel = "MC av")
p4 = Plots.plot(va_mc, va_mc, label = "x = y")
p4 = Plots.plot!(va_mc, out.va, seriestype = :scatter, label = "EP vars", legend = :topleft, xlabel = "MC va")
p5 = Plots.plot(vec(cov_mc), vec(cov_mc), label = "x = y")
p5 = Plots.plot!(vec(cov_mc), vec(cov_ep), seriestype = :scatter, label = "EP cov", legend = :bottomright, xlabel = "MC cross cov")
l = @layout [a b; c d e]
Plots.plot(p1,p2,p3, p4, p5, layout = l)

# (Pv, Ph) = (SpikeAndSlab, ReLu) priors

In [ ]:

# Number of visible and hidden units, Gaussian weigths
#N = convert(Int,28*28*0.5)
#M = convert(Int,400*0.5)
N = 100
M = 10
w = randn(N,M)
y = zeros(N+M)
H = [GaussianEP.TermRBM(w,y,1.0)];
γ = 50.0
ρ = 1.0/21
λ = 50.0
Pv = [GaussianEP.SpikeSlabPrior(ρ, λ, 0.0, 1e-3) for i = 1:N]
Ph = [GaussianEP.ReLUPrior(γ, 0.0, 1e-3, 1e-3) for i = 1:M]


In [ ]:

out = GaussianEP.expectation_propagation(H, Pv, Ph, epsconv = 1e-5, nprint = 1)

cov_ep = zeros(N,M)
for i = 1:N, j = 1:M
	cov_ep[i,j] = out.state.Σ[i,N+j] + out.state.av[i] * out.state.av[N+j]
end


In [ ]:


γ_m = mean([P0[N+i].γ for i = 1:M])
θ_m = mean([P0[N+i].θ for i = 1:M])
λ_m = mean([P0[i].λ for i = 1:N])
println("γ: ",γ_m, " θ: ", θ_m, " λ: ", λ_m)
Pv = [Sampling.SpikeSlabPrior(ρ, λ_m) for i = 1:N]
Ph = [Sampling.ReLUPrior(γ_m, θ_m) for j = 1:M ]
P0mc = vcat(Pv,Ph);

In [ ]:

# Get acceptance rate ~ 0.5
Δ_vec = collect(0.5:0.5:3.0)
acc_tab = Sampling.MC_Δ_scan(Δ_vec,w,P0mc,10^3)
plot(acc_tab[:,1], acc_tab[:,2], xlabel="Δ", ylabel="acc. rate", seriestype = :scatter)

In [ ]:
# Get thermalization time

Δ = 3.5
nblocks = 15
mc_energy = Sampling.MC_t_therm(nblocks, w, P0mc, Δ) 
bl_tab = Sampling.block_anal(nblocks, mc_energy)

p1 = plot(bl_tab[:,1], bl_tab[:,2], ribbon=bl_tab[:,3],fillalpha=.5, xscale =:log) 
plot(p1)

In [ ]:
# Sampling using MCMC
mc_energy = nothing
bl_tab = nothing
mc_out = nothing

Δ = 3.5
Nconf = 10^6
Twait = 10^5

@time mc_out = Sampling.MC_sim(w, P0mc, Twait, Δ; N_iter=Nconf);

In [ ]:
# Compute statistics

@time av_mc, va_mc, cov_mc = Sampling.compute_statistics(mc_out, N, M);

In [ ]:
p1 = heatmap(cov_ep', aspect_ratio=1, ticks= :false, showaxis = :false, xlabel = "visible", ylabel = "hidden" ,title="EP covariance matrix", titlelocation=:center)
p2 = heatmap(cov_mc', aspect_ratio=1, ticks= :false, xlabel = "visible", ylabel = "hidden", showaxis =:false,  title="MC covariance matrix", titlelocation=:center)
p3 = Plots.plot(av_mc, av_mc, label = "x = y") 
p3 = Plots.plot!(av_mc, out.av, seriestype = :scatter, label = "EP means", legend = :topleft, xlabel = "MC av")
p4 = Plots.plot(va_mc, va_mc, label = "x = y")
p4 = Plots.plot!(va_mc, out.va, seriestype = :scatter, label = "EP vars", legend = :topleft, xlabel = "MC va")
p5 = Plots.plot(vec(cov_mc), vec(cov_mc), label = "x = y")
p5 = Plots.plot!(vec(cov_mc), vec(cov_ep), seriestype = :scatter, label = "EP cov", legend = :bottomright, xlabel = "MC cross cov")
l = @layout [a b; c d e]
Plots.plot(p1,p2,p3, p4, p5, layout = l)

# (Pv, Ph) = (Gaussian, Spike and Slab)

In [ ]:

# Number of visible and hidden units, Gaussian weigths
#N = 30
#M = 15
N = 100
M = 10
w = randn(N,M)
y = zeros(N+M)
H = [GaussianEP.TermRBM(w,y,1.0)];
ρ = 0.2
λ = 20.0
μv = 0.5
βv = 50.0

Ph = [GaussianEP.SpikeSlabPrior(ρ, λ) for i = 1:M]
Pv = [GaussianEP.GaussianPrior(μv, βv, 0.0) for i = 1:N];


In [ ]:
out = GaussianEP.expectation_propagation(H, Pv, Ph, nprint = 100);

cov_ep = zeros(N,M)
for i = 1:N, j = 1:M
	cov_ep[i,j] = out.state.Σ[i,N+j] + out.state.av[i] * out.state.av[N+j]
end

In [ ]:

Ph = [Sampling.SpikeSlabPrior(ρ, λ) for i = 1:M]
Pv = [Sampling.GaussianPrior(μv, βv, 0.0) for i = 1:N];
P0mc = vcat(Pv,Ph);

In [ ]:

# Get acceptance rate ~ 0.5
Δ_vec = collect(0.1:0.1:1.2)
acc_tab = Sampling.MC_Δ_scan(Δ_vec,w,P0mc,10^3)
plot(acc_tab[:,1], acc_tab[:,2], xlabel="Δ", ylabel="acc. rate", seriestype = :scatter)

In [ ]:
# Get thermalization time

Δ = 1.2
nblocks = 15
mc_energy = Sampling.MC_t_therm(nblocks, w, P0mc, Δ) 
bl_tab = Sampling.block_anal(nblocks, mc_energy)

p1 = plot(bl_tab[:,1], bl_tab[:,2], ribbon=bl_tab[:,3],fillalpha=.5, xscale =:log) 
plot(p1)

In [ ]:

# Sampling using MCMC
mc_energy = nothing
bl_tab = nothing
mc_out = nothing

Nconf = 5*10^6
Twait = 10^4

mc_out = Sampling.MC_sim(w, P0mc, Twait, Δ; N_iter=Nconf);

In [ ]:
# Compute statistics

av_mc, va_mc, cov_mc = Sampling.compute_statistics(mc_out, N, M);

In [ ]:
p1 = heatmap(cov_ep', aspect_ratio=1, ticks= :false, showaxis = :false, xlabel = "visible", ylabel = "hidden" ,title="EP covariance matrix", titlelocation=:center)
p2 = heatmap(cov_mc', aspect_ratio=1, ticks= :false, xlabel = "visible", ylabel = "hidden", showaxis =:false,  title="MC covariance matrix", titlelocation=:center)
p3 = Plots.plot(av_mc, av_mc, label = "x = y") 
p3 = Plots.plot!(av_mc, out.av, seriestype = :scatter, label = "EP means", legend = :topleft, xlabel = "MC av")
p4 = Plots.plot(va_mc, va_mc, label = "x = y")
p4 = Plots.plot!(va_mc, out.va, seriestype = :scatter, label = "EP vars", legend = :topleft, xlabel = "MC va")
p5 = Plots.plot(vec(cov_mc), vec(cov_mc), label = "x = y")
p5 = Plots.plot!(vec(cov_mc), vec(cov_ep), seriestype = :scatter, label = "EP cov", legend = :bottomright, xlabel = "MC cross cov")
l = @layout [a b; c d e]
Plots.plot(p1,p2,p3, p4, p5, layout = l)